In [1]:
%matplotlib qt
from matplotlib import pyplot as plt

import numpy as np

import sunpy.map
from sunpy.instr.aia import aiaprep
from sunpy.net import Fido, attrs as a

from astropy.coordinates import SkyCoord
from astropy import units as u

from irispy.sji import read_iris_sji_level2_fits

In [2]:
sji = read_iris_sji_level2_fits("iris_l2_20150319_090911_3860359580_SJI_1330_t000.fits")

/Users/tiago/codes/irispy/irispy/sji.py:498: RuntimeWarning: invalid value encountered in sqrt
  + readout_noise.to(u.photon).value**2),


INFO: uncertainty should have attribute uncertainty_type. [astropy.nddata.nddata]


/Users/tiago/codes/irispy/irispy/sji.py:91: UserWarning: This class is still in early stages of development. API not stable.
  warnings.warn("This class is still in early stages of development. API not stable.")


In [3]:
sji


    IRISMapCube
    ---------
    Observatory:		 IRIS
    Instrument:			 SJI
    Bandpass:			 1330.0
    Obs. Start:			 2015-03-19T09:09:11.750000
    Obs. End:			 2015-03-19T14:07:17.532000
    Instance Start:		 2015-03-19T09:09:11.850000
    Instance End:		 2015-03-19T14:07:09.254000
    Total Frames in Obs.:	 1920
    IRIS Obs. id:		 3860359580
    IRIS Obs. Description:	 Large coarse 8-step raster 14x120 8s  C II Deep x 8 Spatial x 2, Spe
    Cube dimensions:		 [1920.  388.  402.] pix
    Axis Types:			 (None, 'custom:pos.helioprojective.lat', 'custom:pos.helioprojective.lon')
    

In [4]:
from sunpy.time import parse_time
erupt_time = parse_time('2015-03-19T12:27:09')
iris_times = sji.extra_coords['TIME']['value']
# Now get the difference between AIA and IRIS times
time_diff = iris_times - erupt_time
time_index = np.argmin(np.abs(time_diff))

In [5]:
plt.figure()
ax = plt.subplot(projection=sji.wcs.dropaxis(-1))
tmp = np.sign(sji[time_index].data) * np.abs(sji[time_index].data) ** 0.3
img = ax.imshow(tmp, cmap='irissji1330', vmin=0.5, vmax=1000**0.3)
ax.coords[0].set_major_formatter('s.s')
ax.coords[1].set_major_formatter('s.s')
ax.grid(color='w', ls=':')



/Users/tiago/codes/irispy/irispy/sji.py:91: UserWarning: This class is still in early stages of development. API not stable.
  warnings.warn("This class is still in early stages of development. API not stable.")


In [6]:
import sunpy.map
from sunpy.instr.aia import aiaprep
from sunpy.net import Fido, attrs as a

In [7]:
result = Fido.search(a.Time('2015-03-19T12:26:40', '2015-03-19T12:28:00'), 
                     a.Instrument("aia"), a.Wavelength(171*u.angstrom), 
                     a.vso.Sample(12*u.second))
result

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2015-03-19 12:27:11,2015-03-19 12:27:12,SDO,AIA,FULLDISK,171.0 .. 171.0
2015-03-19 12:27:23,2015-03-19 12:27:24,SDO,AIA,FULLDISK,171.0 .. 171.0
2015-03-19 12:27:59,2015-03-19 12:28:00,SDO,AIA,FULLDISK,171.0 .. 171.0
2015-03-19 12:26:47,2015-03-19 12:26:48,SDO,AIA,FULLDISK,171.0 .. 171.0
2015-03-19 12:27:47,2015-03-19 12:27:48,SDO,AIA,FULLDISK,171.0 .. 171.0
2015-03-19 12:26:59,2015-03-19 12:27:00,SDO,AIA,FULLDISK,171.0 .. 171.0
2015-03-19 12:27:35,2015-03-19 12:27:36,SDO,AIA,FULLDISK,171.0 .. 171.0


In [13]:
file_download = Fido.fetch(result[0, 3], site='ROB')

[                                        ========================================


In [8]:
# to avoid download again
file_download = ['/Users/tiago/sunpy/data/aia_20150319_122647_0171_image_lev1.fits']  

In [9]:
atmp = sunpy.map.Map(file_download)

In [10]:
aia = aiaprep(atmp)
aia171_rot = aia.rotate(rmatrix=np.matrix(sji.wcs.wcs.pc[:-1, :-1]).I)

/Users/tiago/miniconda/lib/python3.6/site-packages/sunpy/image/transform.py:122: RuntimeWarning: Input data has been cast to float64
  warnings.warn("Input data has been cast to float64", RuntimeWarning)
/Users/tiago/miniconda/lib/python3.6/site-packages/sunpy/map/mapbase.py:1319: Warning: GenericMap.submap now takes pixel values as `bottom_left` and `top_right` not `range_a` and `range_b`
  " and `top_right` not `range_a` and `range_b`", Warning)


In [17]:
plt.figure()
from matplotlib.patches import Rectangle, Polygon

dims = sji.data.shape
_, ypts, xpts = sji.pixel_to_world(np.zeros(4) * u.pix, 
                                   [0, dims[1] - 1, dims[1] -1 , 0] * u.pix,
                                   [0, 0, dims[2] - 1, dims[2] - 1] * u.pix)
iris_points = np.array([xpts.value, ypts.value, ]).T


# Create submap on rotated image, with slightly smaller lower field of view
top_right = SkyCoord(962*u.arcsec, -80*u.arcsec, frame=aia171_rot.coordinate_frame)
bottom_left = SkyCoord(982 * u.arcsec, -600. * u.arcsec, frame=aia171_rot.coordinate_frame)
aia_rot_sub = aia171_rot.submap(top_right, bottom_left)
aia_rot_sub.plot()
# add the same IRIS field-of-view:
ax = plt.gca()
r = Polygon(iris_points, closed=True, edgecolor='y', facecolor='none', lw=2,
            transform=ax.get_transform('world'))
ax.add_patch(r) 
plt.show()

In [18]:
post_time = parse_time('2015-03-19T12:48:25')
time_diff2 = iris_times - post_time
time_index2 = np.argmin(np.abs(time_diff2))

plt.figure()
ax = plt.subplot(projection=sji.wcs.dropaxis(-1))
tmp = np.sign(sji[time_index2].data) * np.abs(sji[time_index2].data) ** 0.3
img = ax.imshow(tmp, cmap='irissji1330', vmin=0.5, vmax=1000**0.3)
ax.coords[0].set_major_formatter('s.s')
ax.coords[1].set_major_formatter('s.s')
ax.grid(color='w', ls=':')

/Users/tiago/codes/irispy/irispy/sji.py:91: UserWarning: This class is still in early stages of development. API not stable.
  warnings.warn("This class is still in early stages of development. API not stable.")


In [24]:
result = Fido.search(a.Time('2015-03-19T12:48:10', '2015-03-19T12:48:40'), 
                     a.Instrument("aia"), a.Wavelength(94*u.angstrom), 
                     a.vso.Sample(12*u.second))
result

Start Time [1],End Time [1],Source,Instrument,Type,Wavelength [2]
,,,,,Angstrom
str19,str19,str3,str3,str8,float64
2015-03-19 12:48:13,2015-03-19 12:48:14,SDO,AIA,FULLDISK,94.0 .. 94.0
2015-03-19 12:48:25,2015-03-19 12:48:26,SDO,AIA,FULLDISK,94.0 .. 94.0
2015-03-19 12:48:37,2015-03-19 12:48:38,SDO,AIA,FULLDISK,94.0 .. 94.0


In [25]:
file_download = Fido.fetch(result[0, 0], site='ROB')

[                                        ========================================


In [19]:
# to avoid download again
file_download = ['/Users/tiago/sunpy/data/aia_20150319_124813_0094_image_lev1.fits']  

In [20]:
atmp = sunpy.map.Map(file_download)
aia94 = aiaprep(atmp)

/Users/tiago/miniconda/lib/python3.6/site-packages/sunpy/image/transform.py:122: RuntimeWarning: Input data has been cast to float64
  warnings.warn("Input data has been cast to float64", RuntimeWarning)
/Users/tiago/miniconda/lib/python3.6/site-packages/sunpy/map/mapbase.py:1319: Warning: GenericMap.submap now takes pixel values as `bottom_left` and `top_right` not `range_a` and `range_b`
  " and `top_right` not `range_a` and `range_b`", Warning)


In [27]:
plt.figure()
aia94.plot()

In [21]:
aia94_rot = aia94.rotate(rmatrix=np.matrix(sji.wcs.wcs.pc[:-1, :-1]).I)

In [31]:
plt.figure()
aia94_rot.plot()

In [22]:
from matplotlib.patches import Rectangle, Polygon

dims = sji.data.shape
_, ypts, xpts = sji.pixel_to_world(np.zeros(4) * u.pix, 
                                   [0, dims[1] - 1, dims[1] -1 , 0] * u.pix,
                                   [0, 0, dims[2] - 1, dims[2] - 1] * u.pix)
iris_points = np.array([xpts.value, ypts.value, ]).T


# Create submap on rotated image, with slightly smaller lower field of view
top_right = SkyCoord(962*u.arcsec, -80*u.arcsec, frame=aia94_rot.coordinate_frame)
bottom_left = SkyCoord(982 * u.arcsec, -600. * u.arcsec, frame=aia94_rot.coordinate_frame)
aia_rot_sub = aia94_rot.submap(top_right, bottom_left)
aia_rot_sub.plot()
# add the same IRIS field-of-view:
ax = plt.gca()
r = Polygon(iris_points, closed=True, edgecolor='y', facecolor='none', lw=2,
            transform=ax.get_transform('world'))
ax.add_patch(r) 
plt.show()

In [54]:
post_time = parse_time('2015-03-19T09:34:21')
time_diff3 = iris_times - post_time
time_index3 = np.argmin(np.abs(time_diff3))

plt.figure()
ax = plt.subplot(projection=sji.wcs.dropaxis(-1))
tmp = np.sign(sji[time_index3].data) * np.abs(sji[time_index3].data) ** 0.3
img = ax.imshow(tmp, cmap='irissji1330', vmin=0.5, vmax=1000**0.3)
ax.coords[0].set_major_formatter('s.s')
ax.coords[1].set_major_formatter('s.s')
ax.grid(color='w', ls=':')

# Plot front of blob
coord1 = [936, -366.6] * u.arcsec
pt, _, _ = sji.pixel_to_world(time_index3 * u.pix, 0 * u.pix, 0 * u.pix)
_, py, px = sji.world_to_pixel(pt, coord1[1], coord1[0])
plt.plot(px, py, 'b+', ms=10)

/Users/tiago/codes/irispy/irispy/sji.py:91: UserWarning: This class is still in early stages of development. API not stable.
  warnings.warn("This class is still in early stages of development. API not stable.")


In [55]:
inc = 4
tmp1 = np.sign(sji[time_index3+inc].data) * np.abs(sji[time_index3+inc].data) ** 0.3
img.set_data(tmp1)
print(iris_times[time_index3])

# New coordinates (by hand)
coord2 = [933.8, -374.2] * u.arcsec
ax.plot(coord2[0].to(u.deg), coord2[1].to(u.deg), 'g+', 
        transform=ax.get_transform('world'), ms=10)

# distance 
dist_arcsec = np.linalg.norm(coord1 - coord2) * u.arcsec

2015-03-19 09:34:20.950000


/Users/tiago/codes/irispy/irispy/sji.py:91: UserWarning: This class is still in early stages of development. API not stable.
  warnings.warn("This class is still in early stages of development. API not stable.")


In [56]:
dist_arcsec

<Quantity 7.91201618 arcsec>

In [57]:
from sunpy.coordinates import frames

# time of first IRIS image
obs_date = sji.extra_coords['TIME']['value'][0]

# get 1" interval at disk-centre between in cartesian coordinates 
a1 = SkyCoord(0 * u.arcsec, 0 * u.arcsec, frame='helioprojective',
              obstime=obs_date).transform_to('heliocentric').x
a2 = SkyCoord(1 * u.arcsec, 0 * u.arcsec, frame='helioprojective',
              obstime=obs_date).transform_to('heliocentric').x
asec_to_Mm = (a2 - a1).to(u.Mm)
asec_to_Mm

<Quantity 0.71864168 Mm>

In [58]:
dist_Mm = dist_arcsec.value * asec_to_Mm
dist_Mm


<Quantity 5.6859046 Mm>

In [65]:
time_delta = sji.extra_coords['TIME']['value'][time_index3 + inc] - sji.extra_coords['TIME']['value'][time_index3]
ts = time_delta.total_seconds() * u.s

vkms = dist_Mm.to(u.km) / ts
vkms

<Quantity 152.68272284 km / s>

In [69]:
post_time = parse_time('2015-03-19T12:33:21')
time_diff4 = iris_times - post_time
time_index4 = np.argmin(np.abs(time_diff4))

plt.figure()
ax = plt.subplot(projection=sji.wcs.dropaxis(-1))
tmp = np.sign(sji[time_index4].data) * np.abs(sji[time_index4].data) ** 0.3
img = ax.imshow(tmp, cmap='irissji1330', vmin=0.5, vmax=1000**0.3)
ax.coords[0].set_major_formatter('s.s')
ax.coords[1].set_major_formatter('s.s')
ax.grid(color='w', ls=':')
print(iris_times[time_index4])

2015-03-19 12:33:21.630000


/Users/tiago/codes/irispy/irispy/sji.py:91: UserWarning: This class is still in early stages of development. API not stable.
  warnings.warn("This class is still in early stages of development. API not stable.")


In [79]:
# my chosen blob
coord1 = [909.3, -375.7] * u.arcsec
ax.plot(coord1[0].to(u.deg), coord1[1].to(u.deg), 'g+', 
        transform=ax.get_transform('world'), ms=10)
inc = 3
tmp1 = np.sign(sji[time_index4+inc].data) * np.abs(sji[time_index4+inc].data) ** 0.3
img.set_data(tmp1)

# by hand
coord2 = [906.2, -384] * u.arcsec
ax.plot(coord2[0].to(u.deg), coord2[1].to(u.deg), 'b+', 
        transform=ax.get_transform('world'), ms=10)

/Users/tiago/codes/irispy/irispy/sji.py:91: UserWarning: This class is still in early stages of development. API not stable.
  warnings.warn("This class is still in early stages of development. API not stable.")


In [80]:
dist_Mm = np.linalg.norm(coord1 - coord2) * asec_to_Mm
time_delta = sji.extra_coords['TIME']['value'][time_index4 + inc] - sji.extra_coords['TIME']['value'][time_index4]
ts = time_delta.total_seconds() * u.s

vkms = dist_Mm.to(u.km) / ts
vkms

<Quantity 227.88767026 km / s>